# AWS IoT - Subscribing to External AWS IoT Account Topics 


Now we are going to subscribe our sensors topic to start receiving real data. We are going to use a temporary certificate to access the MQTT topic during this workshop.

### Let's recover some data and stablish a MQTT connection with our own AWS Account & thing

In [3]:
#Hour and minute when this started to create unique names
%store -r HM
#Home directory of the user running this notebook
%store -r home
#Private key file name, for AWS IoT authentication
%store -r private_key
#Certificate file name, for AWS IoT authentication
%store -r certificate_file
#Certificate authority file name, for AWS IoT authentication
%store -r ca_file
#Address to connect using MQTT
%store -r endpoint_address

from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

client_id="NotebookCient{}".format(HM)
mqtt = AWSIoTMQTTClient(client_id)
mqtt.configureEndpoint(endpoint_address, 8883)
mqtt.configureCredentials(ca_file, private_key, certificate_file)
mqtt.configureConnectDisconnectTimeout(10)
mqtt.connect()
mqtt

In [4]:

! mkdir -p "{home}/aws_iot_hub"

hub_certificate_file = "{}/aws_iot_hub/certificate-hub.pem".format(home)
hub_public_key = "{}/aws_iot_hub/public-key-hub.pem".format(home)
hub_private_key = "{}/aws_iot_hub/private-key-hub.pem".format(home)
hub_ca_file = "{}/aws_iot_hub/ca.pem".format(home)

%store hub_certificate_file
%store hub_public_key
%store hub_private_key
%store hub_ca_file


Stored 'hub_certificate_file' (str)
Stored 'hub_public_key' (str)
Stored 'hub_private_key' (str)
Stored 'hub_ca_file' (str)


In [ ]:
! wget https://s3.amazonaws.com/healthcare-workshop/certificate-hub.pem -O {hub_certificate_file}
! wget https://s3.amazonaws.com/healthcare-workshop/public-key-hub.pem -O {hub_public_key}
! wget https://s3.amazonaws.com/healthcare-workshop/private-key-hub.pem -O {hub_private_key}
! wget https://www.symantec.com/content/en/us/enterprise/verisign/roots/VeriSign-Class%203-Public-Primary-Certification-Authority-G5.pem -O {hub_ca_file}


Double check that you have the 4 files required for TLS mutual authentication used by AWS IoT
1. Private Key
1. Public Key
1. Certificate
1. Root CA

In [5]:
! printf "Private Key {hub_private_key} " && [ -f {hub_private_key} ] && printf "found" || printf "not found"
! printf "\nPublic Key {hub_public_key} " && [ -f {hub_public_key} ] && printf "found" || printf "not found"
! printf "\nCertificate {hub_certificate_file} " && [ -f {hub_certificate_file} ] && printf "found" || printf "not found"
! printf "\nCertification Authority {hub_ca_file} " && [ -f {hub_ca_file} ] && printf "found" || printf "not found"

Private Key /home/ec2-user/aws_iot_hub/private-key-hub.pem found
Public Key /home/ec2-user/aws_iot_hub/public-key-hub.pem found
Certificate /home/ec2-user/aws_iot_hub/certificate-hub.pem found
Certification Authority /home/ec2-user/aws_iot_hub/ca.pem found

# Connecting to our AWS IoT Account  - Workshop Data Hub

AWS IoT provides account-specific endpoints for your things to connect. We are using the instructor workshop endpoint at this time:

In [6]:
hub_endpoint_address = "a2enzbgohiblz2.iot.us-east-1.amazonaws.com"
%store hub_endpoint_address
hub_endpoint_address

Stored 'hub_endpoint_address' (str)


'a2enzbgohiblz2.iot.us-east-1.amazonaws.com'

Ready to go! Create your MQTT client instance:

In [7]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

client_id="NotebookCient{}".format(HM)
mqtt_hub = AWSIoTMQTTClient(client_id)
mqtt_hub.configureEndpoint(hub_endpoint_address, 8883)
mqtt_hub.configureCredentials(hub_ca_file, hub_private_key, hub_certificate_file)
mqtt_hub.configureConnectDisconnectTimeout(10)
mqtt_hub.connect()
mqtt_hub

# Forward Data Hub MQTT messages to our Account / Thing

In [10]:
hub_topic_name_in ="healthcare/bloodpressure"
my_topic_out = "myhealthcare"
%store hub_topic_name_in

subscribe_qos = 1
publish_qos = 0

def custom_callback(client, userdata, message):
    print("\nReceived a new message: \n")
    print(message.payload.decode("utf-8") )
    mqtt.publish(my_topic_out, message.payload.decode("utf-8"), publish_qos)
    print("\n--------------\n")
    
mqtt_hub.subscribe(hub_topic_name_in, subscribe_qos, custom_callback)

Stored 'hub_topic_name_in' (str)


True


Received a new message: 

{ "on" : true}

--------------



Now are going to forward the received message from Data Hub to our Thing in our account:

In [ ]:
subscribe_qos = 1
publish_qos = 0
pub_topic_name="myHealthcare"
def custom_callback1(client, userdata, message):
    print("\nReceived a new message: \n")
    print(message.payload.decode("utf-8").decode("utf-8") )
    mqtt.publish(pub_topic_name, message.payload, publish_qos)
    print("\n--------------\n")
    
mqtt_hub.subscribe(hub_topic_name, subscribe_qos, custom_callback1)

All working fine, done for now, disconnect.

In [ ]:
mqtt.disconnect()
mqtt_hub.disconnect()


Now we don't want to loose this data, right?
Let's proceed to the [Storing Telemetry Data](aws-iot-storing-telemetry.ipynb) notebook right now!